In [1]:
from robin_stocks import robinhood as r
import pandas as pd
from datetime import datetime
import logging
# import traceback # for debugging
from sys import exit # for debugging
import traceback # for debugging
import time
from os import path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from tqdm import tqdm
import numpy as np
from icecream import ic
from colorama import Fore, Back, Style
# Login to Robinhood
r.login('username', 'password')
panic_mode = False # set to True to sell all crypto positions immediately
# List of cryptocurrencies to trade
cryptos = ['BTC', 'ETH', 'ADA', 'DOGE', 'MATIC', 'SHIB', 'ETC', 'UNI', 'AAVE', 'LTC', 'LINK', 'COMP', 'USDC', 'SOL', 'AVAX', 'XLM', 'BCH', 'XTZ']

# log file
logger = logging.getLogger('crypto_trader')
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('crypto_trader.log')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
# import datetime now
from datetime import datetime
# import now function
from datetime import datetime
# import time
import time

now = datetime.now()

# Initialize an empty dataframe to store crypto data
crypto_data = pd.DataFrame()

def get_crypto_data(cryptos, crypto_data):

    for crypto in tqdm(cryptos):
        try:
            #print(f'Getting data for {crypto}, please wait...',end='')
            # Get historical data
            historicals = r.get_crypto_historicals(crypto, interval='day', span='week')
            current_price = historicals[-1]['close_price']
            historical_prices = [x['close_price'] for x in historicals]

            # Load account profile
            profile = r.profiles.load_account_profile()
            buying_power = profile['buying_power']

            # Get crypto positions
            positions = r.crypto.get_crypto_positions()
            print(f'found {len(positions)} positions.')
            for position in tqdm(positions):
                # print(position)
                if position['currency']['code'] == crypto:
                    pos_dict = position['currency']
                    min_order_size = float(pos_dict['increment'])
                    coin_holdings = float(position['quantity_available'])


            # Get profile data
            profile_data = r.profiles.load_portfolio_profile()
            current_equity = profile_data['equity']
            current_equity = float(current_equity)
            previous_equity = profile_data['adjusted_equity_previous_close']
            previous_equity = float(previous_equity)
            daily_profit = float(current_equity) - float(previous_equity)

            # convert all the numerical values in historical_prices to float
            historical_prices = [float(x) for x in historical_prices]

            # Append data to dataframe
            crypto_data = crypto_data.append({
                'ticker': crypto,
                'current_price': current_price,
                'historical_prices': historical_prices,
                'buying_power': buying_power,
                'min_order_size': min_order_size,
                'coin_holdings': coin_holdings,
                'updated_at': now,
                'current_equity': current_equity,
                'previous_equity': previous_equity,
                'daily_profit': daily_profit
            }, ignore_index=True)
        except Exception as e:
            print(f'Error getting data for {crypto}: {e}')
            logger.error(f'Error getting data for {crypto}: {e}')
            traceback.print_exc()
            pass

    def add_me(crypto_data, metric, metric_name):
        if isinstance(metric, pd.Series):
            # if rsi is a series, get the last value
            crypto_data.loc[index, metric_name] = metric.iloc[-1]
        elif isinstance(metric, float):
            # if rsi is a float, just use it
            crypto_data.loc[index, metric_name] = metric
        elif isinstance(metric, np.ndarray):
            # if rsi is an array, get the last value
            crypto_data.loc[index, metric_name] = metric[-1]
        else:
            # otherwise, just use the value
            crypto_data.loc[index, metric_name] = metric
        return crypto_data

    # Calculate RSI, MACD, Bollinger Bands, etc. and add to dataframe
    for index, row in tqdm(crypto_data.iterrows()):
        prices = pd.Series(row['historical_prices'])

        # Calculate RSI (Relative Strength Index)
        # first make a copy of the data frame twice
        up_df, down_df = prices.copy(), prices.copy()
        # For up days, if the price is lower than previous price, set price to 0.
        up_df[up_df < up_df.shift(1)] = 0
        # For down days, if the price is higher than previous price, set price to 0.
        down_df[down_df > down_df.shift(1)] = 0
        # We need change and average gain for the first calculation.
        # Calculate the 1st-day change and average gain
        up_df.iloc[0] = 0
        down_df.iloc[0] = 0
        # Calculate the EWMA (Exponential Weighted Moving Average)
        roll_up1 = up_df.ewm(span=14).mean()
        roll_down1 = down_df.ewm(span=14).mean()
        # Calculate the RSI based on EWMA
        RS1 = roll_up1 / roll_down1
        RSI1 = 100.0 - (100.0 / (1.0 + RS1))
        rsi = RSI1 # save it for later
        
        # Calculate MACD (Moving Average Convergence Divergence)
        exp1 = prices.ewm(span=12, adjust=False).mean()
        # Calculate the 26-day EMA
        exp2 = prices.ewm(span=26, adjust=False).mean()
        # Subtract 26-day EMA from 12-day EMA, and you will get the first MACD
        macd = exp1 - exp2
        # Calculate the 9-day EMA of the MACD above
        exp3 = macd.ewm(span=9, adjust=False).mean()
        # Plot the MACD and exp3
        macd = macd - exp3
        # save it for later
        macd = macd.iloc[-1]
        macd_signal = exp3.iloc[-1] # the signal line
        macd_hist = macd - macd_signal # the histogram
        ma200 = prices.rolling(200).mean().iloc[-1] # 200 day moving average
        ma50 = prices.rolling(50).mean().iloc[-1] # 50 day moving average
        ma20 = prices.rolling(20).mean().iloc[-1] # 20 day moving average
        sma = prices.rolling(5).mean().iloc[-1] # 5 day simple moving average
        ema = prices.ewm(span=5, adjust=False).mean().iloc[-1] # 5 day exponential moving average

        # Calculate Bollinger Bands
        # Calculating the Upper and Lower Bands
        # We will be using a 14 day window
        window = 14
        # Calculate rolling mean and standard deviation using number of days set above
        rolling_mean = prices.rolling(window).mean()
        rolling_std = prices.rolling(window).std()
        # Create two new DataFrame columns to hold values of upper and lower Bollinger bands
        upper_band = pd.Series()
        upper_band = rolling_mean + (rolling_std * 2)
        lower_band = rolling_mean - (rolling_std * 2)

        # Add to dataframe
        print(f'Adding technical indicators for {row["ticker"]} to dataframe\n\t...')
        #^ Add the RSI
        crypto_data = add_me(crypto_data, rsi, 'rsi')
        #^ Add the MACD
        crypto_data = add_me(crypto_data, macd, 'macd')
        #^ Add the SMA from the Bollinger Bands
        crypto_data = add_me(crypto_data, rolling_mean, 'sma')
        #^ Add the EMA from the Bollinger Bands
        crypto_data = add_me(crypto_data, ema, 'ema')
        #^ Add the 200 day moving average
        crypto_data = add_me(crypto_data, ma200, 'ma200')
        #^ Add the 50 day moving average
        crypto_data = add_me(crypto_data, ma50, 'ma50') 
        #^ Add the 20 day moving average
        crypto_data = add_me(crypto_data, ma20, 'ma20')
        #^ Add the 5 day simple moving average
        crypto_data = add_me(crypto_data, sma, 'sma')
        #^ Add the 5 day exponential moving average
        crypto_data = add_me(crypto_data, ema, 'ema')
        #^ Add the MACD signal line
        crypto_data = add_me(crypto_data, macd_signal, 'macd_signal')
        #^ Add the MACD histogram
        crypto_data = add_me(crypto_data, macd_hist, 'macd_hist')
        #^ Add the Upper Band
        crypto_data = add_me(crypto_data, upper_band, 'upper_band')
        #^ Add the Lower Band
        crypto_data = add_me(crypto_data, lower_band, 'lower_band')
    # save the crypto_data dataframe to a csv file
    crypto_data.to_csv('crypto_data.csv', index=False)
    return crypto_data


### Part 2